# DATA PRE-PROCESSING

In [10]:
import pandas as pd

In [11]:
nlcd_data = "/s/chopin/e/proj/sustain/sapmitra/kubernetes_project/nlcd_vectors/nlcds_CO.csv"

df_str = pd.read_csv(nlcd_data,header=None, dtype=str)
df = pd.read_csv(nlcd_data,header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2310011023,0.0,0.010085,0.0,0.010417,0.002621,0.000100,0.000012,0.001092,0.001803,...,0.0,0.748177,0.041141,0.0,0.0,0.0,0.015899,0.022232,0.005588,0.002077
1,2132331231,0.0,0.015977,0.0,0.000000,0.000000,0.000000,0.000000,0.000011,0.000265,...,0.0,0.000000,0.926565,0.0,0.0,0.0,0.000030,0.002654,0.000678,0.053772
2,2310123121,0.0,0.000082,0.0,0.005974,0.001025,0.000046,0.000000,0.000012,0.000326,...,0.0,0.085851,0.857291,0.0,0.0,0.0,0.000000,0.002380,0.001080,0.015015
3,2132232120,0.0,0.000466,0.0,0.006934,0.006253,0.001159,0.000056,0.020026,0.000015,...,0.0,0.934513,0.029372,0.0,0.0,0.0,0.000352,0.000000,0.000197,0.000496
4,2310301112,0.0,0.000012,0.0,0.001620,0.000057,0.000000,0.000000,0.000000,0.000000,...,0.0,0.168714,0.827650,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3470,2310210123,0.0,0.000029,0.0,0.008223,0.001782,0.000077,0.000000,0.000019,0.000000,...,0.0,0.575560,0.262437,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000270
3471,2310122123,0.0,0.001298,0.0,0.026021,0.021089,0.003962,0.000185,0.000029,0.000000,...,0.0,0.327384,0.015714,0.0,0.0,0.0,0.090190,0.392848,0.036662,0.084412
3472,2310300101,0.0,0.000003,0.0,0.003186,0.001479,0.000003,0.000000,0.000000,0.000055,...,0.0,0.642766,0.004173,0.0,0.0,0.0,0.000000,0.000000,0.000193,0.000354
3473,2310003201,0.0,0.007022,0.0,0.012103,0.003901,0.000478,0.000005,0.047941,0.000024,...,0.0,0.765582,0.004372,0.0,0.0,0.0,0.057421,0.051812,0.007381,0.006283


In [12]:
quad_keys = df_str.loc[:,0]
nlcd_vectors = df.loc[:, range(1,22)]
print(nlcd_vectors.shape, quad_keys.shape)

(3475, 21) (3475,)


# TRAINING

In [13]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, init="k-means++")#.fit(nlcd_vectors)

In [14]:
'''from sklearn.metrics import silhouette_score
kmeans.labels_
s = silhouette_score(nlcd_vectors, kmeans.labels_, metric = 'euclidean')
s'''

"from sklearn.metrics import silhouette_score\nkmeans.labels_\ns = silhouette_score(nlcd_vectors, kmeans.labels_, metric = 'euclidean')\ns"

In [15]:
alldistances = kmeans.fit_transform(nlcd_vectors)
centroids = kmeans.cluster_centers_
alldistances

array([[0.56034423, 0.97455015, 0.10426408, 0.95046033],
       [1.01573199, 0.2289788 , 1.15398545, 0.87457441],
       [0.91968899, 0.14917205, 1.04292955, 0.82419693],
       ...,
       [0.38402216, 0.98706121, 0.33623935, 0.95612524],
       [0.64322236, 1.00616752, 0.10866809, 0.94895967],
       [0.52726857, 0.9315202 , 0.71609032, 0.86273801]])

# POST TRAINING

In [16]:
alldistances.shape

(3475, 4)

In [17]:
predicted_clusters = kmeans.predict(nlcd_vectors)

In [18]:
predicted_clusters

array([2, 1, 1, ..., 2, 2, 0], dtype=int32)

In [19]:
p = predicted_clusters
p_df = pd.DataFrame(p)

In [20]:
q = quad_keys
q_df = q.to_frame()

In [21]:
#cons = pd.concat([q_df, p_df], axis = 1)
#cons

In [22]:
#cons.to_csv("/s/chopin/e/proj/sustain/sapmitra/kubernetes_project/nlcd_vectors/nlcd_clusters.csv", header=None, index = None)

In [23]:
cluster_map = {}
cluster_map_lite = {}
for index, row in q_df.iterrows():
    qhash = str(row[0])
    pred = p_df.iloc[index][0]
    dist = alldistances[index][pred]
    
    if pred not in cluster_map:
        elements = []
        elements_lite = []
        cluster_map[pred] = elements
        cluster_map_lite[pred] = elements_lite
    else:
        elements = cluster_map[pred]
        elements_lite = cluster_map_lite[pred]
    elements.append({"qhash":qhash, "dist":dist})
    elements_lite.append(qhash)
    #print(qhash, pred, dist)

cluster_map_lite

{2: ['02310011023',
  '02132232120',
  '02310311223',
  '02132233000',
  '02132223321',
  '02310213131',
  '02132223002',
  '02310132013',
  '02132232131',
  '02310212031',
  '02310123012',
  '02310032200',
  '02132221320',
  '02132223010',
  '02310010023',
  '02310011101',
  '02310021231',
  '02310021003',
  '02310300120',
  '02310023310',
  '02310032030',
  '02310033302',
  '02310032311',
  '02310201133',
  '02310122300',
  '02132231312',
  '02310021322',
  '02310032233',
  '02310012301',
  '02310032232',
  '02310310312',
  '02310211202',
  '02310023010',
  '02310010113',
  '02132223023',
  '02310213100',
  '02132231211',
  '02132233032',
  '02311022222',
  '02310302011',
  '02310023131',
  '02310021101',
  '02132231232',
  '02310032023',
  '02310023101',
  '02310003022',
  '02310210133',
  '02310301213',
  '02310023123',
  '02132232213',
  '02311202031',
  '02310310321',
  '02310023211',
  '02310032012',
  '02310211302',
  '02310012003',
  '02310311321',
  '02132232333',
  '02132223

In [24]:
len(cluster_map_lite[1])

865

In [25]:
cluster_centroid_map = {}
for key in cluster_map:
    elements = cluster_map[key]
    mini = 999
    mini_key = None
    for element in elements:
        if element['dist'] < mini:
            mini_key = element['qhash']
    cluster_centroid_map[key] = mini_key
len(cluster_centroid_map)

4

In [26]:
import pickle
save_path = "/s/chopin/e/proj/sustain/sapmitra/kubernetes_project/TransferLearning/mapz/"
with open(save_path+"cluster_lite", 'wb') as handle:
    pickle.dump(cluster_map_lite, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(save_path+"cluster_centroids", 'wb') as handle:
    pickle.dump(cluster_centroid_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
centroids = "mapz/cluster_centroids"
all_nodes = "mapz/cluster_lite"
with open(centroids, 'rb') as fp:
    centroid_map = pickle.load(fp)
with open(all_nodes, 'rb') as fp:
    nodes_map = pickle.load(fp)

In [28]:
print(nodes_map)

{2: ['02310011023', '02132232120', '02310311223', '02132233000', '02132223321', '02310213131', '02132223002', '02310132013', '02132232131', '02310212031', '02310123012', '02310032200', '02132221320', '02132223010', '02310010023', '02310011101', '02310021231', '02310021003', '02310300120', '02310023310', '02310032030', '02310033302', '02310032311', '02310201133', '02310122300', '02132231312', '02310021322', '02310032233', '02310012301', '02310032232', '02310310312', '02310211202', '02310023010', '02310010113', '02132223023', '02310213100', '02132231211', '02132233032', '02311022222', '02310302011', '02310023131', '02310021101', '02132231232', '02310032023', '02310023101', '02310003022', '02310210133', '02310301213', '02310023123', '02132232213', '02311202031', '02310310321', '02310023211', '02310032012', '02310211302', '02310012003', '02310311321', '02132232333', '02132223013', '02132230333', '02132230301', '02132233110', '02310032213', '02132233123', '02310003300', '02132231300', '0231